In [ ]:
!pip install python-docx #<-- Yes you can directly install in Colab
!pip install textract
import textract
import docx
from google.colab import files
from docx import Document
import codecs
import re
import json

In [ ]:
# Преобразование word to txt

filepath = r"Протокол совещания_24.04.11.docx"
document = Document(filepath)

with codecs.open('status_1.txt', 'w', "utf-8-sig") as o_file:
    for para in document.paragraphs:
        text = re.sub(r'\n{2,}', '\n', para.text)
        o_file.write(text + '\n')

In [ ]:
# Удаляем весь текст до части "РЕШИЛИ"
def save_text_after_keyword(input_file, keyword, output_file):
    with open(input_file, 'r') as f:
        lines = f.readlines()

    found = False
    text_after_keyword = []

    for line in lines:
        if found:
            text_after_keyword.append(line)
        elif keyword in line:
            found = True

    with open(output_file, 'w') as f:
        f.writelines(text_after_keyword)

# Пример использования
input_file = "status_1.txt"
keyword = "РЕШИЛИ:"
output_file = "status_2.txt"

save_text_after_keyword(input_file, keyword, output_file)


In [ ]:
# Чистим подготовленный txt файл от переносов
# Открываем файл и считываем его содержимое
with open('status_2.txt', 'r') as file:
    text = file.read()

# Заменяем два и более переноса строк на один
text = re.sub(r'\n{2,}', '\n', text)

# Записываем измененный текст обратно в файл
with open('status_3.txt', 'w') as file:
    file.write(text)


In [ ]:
def process_file(file_path, output_file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        i = 0
        while i < len(lines):
            line = lines[i].strip()

            if line.startswith("В части Проекта"):
                output_file.write("Заголовок: " + line + "\n")
                i += 1
                if i < len(lines) and lines[i].strip().startswith("Ответственный:"):
                    # Нет дополнительных строк между
                    pass
                elif (i + 1 < len(lines) and
                      not lines[i+1].strip().startswith("Ответственный:")):
                    # Две строки до "Ответственный:"
                    output_file.write("Подзаголовок: " + lines[i].strip() + "\n")
                    i += 1
                    output_file.write("Задача: " + lines[i].strip() + "\n")
                elif i < len(lines) and not lines[i].strip().startswith("Ответственный:"):
                    # Одна строка до "Ответственный:"
                    output_file.write("Задача: " + lines[i].strip() + "\n")

            elif line.startswith("Срок"):
                output_file.write(line + "\n")
                i += 1
                if i < len(lines) and lines[i].strip().startswith("В части Проекта"):
                    # Возвращаемся к нормальной обработке с "В части Проекта"
                    continue
                elif i < len(lines) and not lines[i].strip().startswith("Ответственный:"):
                    next_line = lines[i].strip()
                    if (i + 1 < len(lines) and
                        not lines[i+1].strip().startswith("Ответственный:")):
                        output_file.write("Подзаголовок: " + next_line + "\n")
                        i += 1
                        output_file.write("Задача: " + lines[i].strip() + "\n")
                    else:
                        output_file.write("Задача: " + next_line + "\n")

            elif line.startswith("Ответственный:"):
                output_file.write(line + "\n")

            else:
                output_file.write(line + "\n")

            i += 1

filename = 'status_3.txt'
output_filename = 'status_4.txt'
process_file(filename, output_filename)

In [ ]:
def parse_tasks(text):
    projects = {}
    current_project = None
    current_subtitle = None

    lines = text.strip().split('\n')
    for line in lines:
        if line.startswith('Заголовок:'):
            current_project = re.search(r'Заголовок: (.+)', line).group(1)
            projects[current_project] = []
        elif line.startswith('Подзаголовок:'):
            current_subtitle = re.search(r'Подзаголовок: (.+)', line).group(1)
            projects[current_project].append({current_subtitle: []})
        elif line.startswith('Задача:'):
            task = re.search(r'Задача: (.+)', line).group(1)
            responsible_match = re.search(r'Ответственный: (.+)', lines[lines.index(line) + 1])
            responsible = responsible_match.group(1) if responsible_match else None
            deadline_match = re.search(r'Срок: (.+)', lines[lines.index(line) + 2])
            deadline = deadline_match.group(1) if deadline_match else None
            if current_subtitle:
                projects[current_project][-1][current_subtitle].append({
                    "Задача": task,
                    "Ответственный": responsible,
                    "Срок": deadline
                })
            else:
                projects[current_project].append({
                    "Задача": task,
                    "Ответственный": responsible,
                    "Срок": deadline
                })

    return projects

if __name__ == "__main__":
    with open('status_4.txt', 'r', encoding='utf-8') as file:
        text = file.read()

    parsed_data = parse_tasks(text)
    with open('status_5.json', 'w', encoding='utf-8') as json_file:
        json.dump(parsed_data, json_file, indent=2, ensure_ascii=False)

In [32]:
import json

def process_data(data):
    with open('status_6.txt', 'w', encoding='utf-8') as txt_file:
        for section_title, tasks in data.items():
            txt_file.write(f"{section_title}\n")
            for task in tasks:
                if isinstance(task, dict):
                    task = [task]
                if "Задача" in task[0]:
                    for t in task:
                        txt_file.write(f"<br>Задача: {t['Задача']}\n")
                        txt_file.write("---\n")
                        txt_file.write(f"Ответственный: {t['Ответственный']}\n")
                        txt_file.write("---\n")
                        txt_file.write(f"Срок: {t['Срок']} {t.get('Дата', '')}\n")

                        txt_file.write("\n")
                else:
                    for sub_title, sub_tasks in task[0].items():
                        txt_file.write(f"{sub_title}\n")
                        for t in sub_tasks:
                            txt_file.write(f"<br>Задача: {t['Задача']}\n")
                            txt_file.write("---\n")
                            txt_file.write(f"Ответственный: {t['Ответственный']}\n")
                            txt_file.write("---\n")
                            txt_file.write(f"Срок: {t['Срок']} {t.get('Дата', '')}\n")
                            txt_file.write("\n")

### Перегрупировка json для ответственного

In [ ]:
## todo - правильно указывать Подпроект

In [ ]:
import json

def reformat_json(input_json):
    output_json = {}

    for project, tasks in input_json.items():
        for task in tasks:
            if isinstance(task, dict):
                responsible = task.get("Ответственный", "-")
                deadline = task.get("Срок", "-")
                task_name = task.get("Задача", "-")
                project_name = project
                subproject_name = "-"

                task_obj = {
                    "Задача": task_name,
                    "Срок": deadline,
                    "Проект": project,
                    "Подпроект": "-"
                }

                if isinstance(list(task.values())[0][0], dict):
                    subproject_name = list(task.keys())[0]
                    project_name = project.split("В части Проекта ")[-1]
                    task_obj["Проект"] = project_name
                    task_obj["Подпроект"] = subproject_name

                output_json[responsible] = output_json.get(responsible, []) + [task_obj]

            elif isinstance(task, list):
                for subproject, sub_tasks in task[0].items():
                    for sub_task in sub_tasks:
                        responsible = sub_task.get("Ответственный", "-")
                        deadline = sub_task.get("Срок", "-")
                        task_name = sub_task.get("Задача", "-")
                        project_name = project
                        subproject_name = subproject

                        task_obj = {
                            "Задача": task_name,
                            "Срок": deadline,
                            "Проект": project,
                            "Подпроект": subproject_name
                        }

                        output_json[responsible] = output_json.get(responsible, []) + [task_obj]

    return output_json


with open('status_5.json', 'r', encoding='utf-8') as file:
    input_json = json.load(file)

reformatted_json = reformat_json(input_json)
print(json.dumps(reformatted_json, indent=2, ensure_ascii=False))


{
  "Д.В. Громак": [
    {
      "Задача": "Обеспечить получение от АО «ВНИИГ» получение информации по срокам предоставления объемов демонтажа на основании данных по фундаменту, полученных от АО «УТЗ».",
      "Срок": "19.04.2024 (05.04.2024)",
      "Проект": "В части Проекта КОММод:",
      "Подпроект": "-"
    },
    {
      "Задача": "Направить в адрес ПАО «ТГК-1» комплект рабочей документации по тепловой изоляции турбоагрегата (изм.0).",
      "Срок": "15.04.2024 (31.03.2024)",
      "Проект": "В части Проекта КОММод:",
      "Подпроект": "-"
    },
    {
      "Задача": "Принять оформление внесенных в чертежи изменений в форме «облака».",
      "Срок": "\tпостоянно",
      "Проект": "В части Проекта КОММод:",
      "Подпроект": "-"
    },
    {
      "Задача": "Обеспечить получение ТКП на систему возбуждения от НПО Электромаш.",
      "Срок": "15.04.2024",
      "Проект": "В части Проекта КОММод:",
      "Подпроект": "-"
    },
    {
      "Задача": "Проработать техническое решен

In [ ]:
#todo - вывод таблицы ответственные

In [ ]:
#todo my sql

In [ ]:
#todo редактирование